Code of activity source extraction from Ca2+ imaging data time stacks

@uthor: Raju Tomer.

In [1]:
import os, time
import skimage.external.tifffile as tff
import numpy as np
import matplotlib.pyplot as plt
import caiman as cm
from caiman.utils.visualization import plot_contours, nb_view_patches, nb_plot_contour, inspect_correlation_pnr

In [ ]:
# Load data
def load_pickle(d_dir,fn):
    pickle_in = open(os.path.join(d_dir,fn),'rb')
    dict_d = pickle.load(pickle_in)
    pickle_in.close()
    return dict_d


prefix_swap_sw = False

## Batch 07
data_dir = r'Y:\Data\C57'
f_data_dir = r'Y:\Data\C57\Results'

dict_f_NsP = load_pickle(f_data_dir, r'dict_f_Sph.pickle')
dict_im_lab = load_pickle(f_data_dir,  r'dict_im_lab.pickle')
dict_in_fn = load_pickle(f_data_dir, r'dict_in_fn.pickle')

print(dict_in_fn[0])
if (prefix_swap_sw):
    for key in dict_in_fn.keys():
        max_file_name = os.path.basename(dict_in_fn[key])
        div = os.path.basename(os.path.dirname(dict_in_fn[key]))
        dict_in_fn[key] = os.path.join(data_dir, div, max_file_name)
        print(dict_in_fn[key])

In [ ]:
tmp_dir = r'E:\Raju\NSp\tmp_dir'
analysis_dir = r'X:\People\Raju\Data\C57\LSTM'

In [ ]:
to_run = list(dict_in_fn.keys())
print(to_run)
# to_run = to_run[0:10]
# print(to_run)


In [ ]:
def run_caiman(Y, dview, n_processes=24, frate = 30, gSig = 3, rf = 80):
    print(Y.shape)
    decay_time = 0.8                 
    p = 1               
    K = None            
    gSiz = 4*gSig + 1 
    stride_cnmf = gSiz + 5
    merge_thresh = .7   
    tsub = 1            
    ssub = 1            
    Ain = None          
    low_rank_background = None
    gnb = -1         
    nb_patch = -1       
    min_corr = .8
    min_pnr = 10
    ssub_B = 1
    ring_size_factor = 1.4
    min_SNR = 3           
    r_values_min = 0.85   
    cnm = cnmf.CNMF(n_processes=n_processes,
                    method_init='corr_pnr',             
                    k=K,
                    gSig=(gSig, gSig),
                    gSiz=(gSiz, gSiz),
                    merge_thresh=merge_thresh,
                    p=p,
                    dview=dview,
                    tsub=tsub,
                    ssub=ssub,
                    Ain=Ain,
                    rf=rf,
                    stride=stride_cnmf,
                    only_init_patch=True,               
                    gnb=gnb,
                    nb_patch=nb_patch,
                    method_deconvolution='oasis',       
                    low_rank_background=low_rank_background,
                    update_background_components=True,  
                    min_corr=min_corr,
                    min_pnr=min_pnr,
                    normalize_init=False,               
                    center_psf=True,                    
                    ssub_B=ssub_B,
                    ring_size_factor=ring_size_factor,
                    del_duplicates=True,                
                    border_pix=0)                 
    print('Running cnm')
    cnm.fit(Y)
    return cnm

In [ ]:
get_ipython().magic(u'matplotlib inline')
print(to_run)
for key in to_run:
    f = open(os.path.join(analysis_dir,"error_log.txt"), "a")
    tic = time.clock()
    print(key,dict_in_fn[key])
    im_lab = dict_im_lab[key]
    im_fname = dict_in_fn[key]
    im_fname = im_fname.replace('MAX_','')
    print(im_fname)
    im = tff.imread(im_fname)
    print('im', im.shape)
    im[im==0] = 10
    if (os.path.exists(dict_in_fn[key])):
        im_max = tff.imread(dict_in_fn[key])
    else:
        im_max = np.max(im,axis=0)
    fig = plt.figure(figsize=(8,4))
    tmp_tmp_dir = os.path.join(tmp_dir, 'key_' + str(key) + '_' + str(np.random.random()))
    os.mkdir(tmp_tmp_dir)
    tmp_fname = os.path.join(tmp_tmp_dir, 'memfile_tmp' + str(np.random.random()) + '.tif')
    tff.imsave(tmp_fname, im)
    fname_new = cm.save_memmap([tmp_fname], base_name='memmap_', order = 'C')
    if os.path.isfile(tmp_fname):
        os.remove(tmp_fname)
    else:
        print("Temp File not found")
        
    Yr, dims, T = cm.load_memmap(fname_new)
    Y = Yr.T.reshape((T,) + dims, order='F')
        
    print('Generating Corr Image:')
    gSig = 3
    try:
        if 'dview' in locals():
            cm.stop_server(dview=dview)
        c, dview, n_processes = cm.cluster.setup_cluster(
            backend='local', n_processes=None, single_thread=False)
        cn_filter, pnr = cm.summary_images.correlation_pnr(Y, gSig=gSig, swap_dim=False) # change swap dim if output looks weird, it is a problem with tiffile
        inspect_correlation_pnr(cn_filter, pnr)
        plt.show()
        print('Running cnm')
        cnm = run_caiman(Y, dview=dview, n_processes=n_processes, frate = 30, gSig = gSig, rf = 40)
        cm.stop_server(dview=dview)
    except:
        try:
            cm.stop_server(dview=dview) # stop it if it was running
        except:
            pass
        print('Error in', key)
        f.write('\nError in key ' + str(key))
        f.close()
        continue
    try:
        nb_view_patches(Yr, cnm.estimates.A.tocsc(), cnm.estimates.C,
            cnm.estimates.b, cnm.estimates.f, dims[0], dims[1], YrA=cnm.estimates.YrA, image_neurons=cn_filter,
            denoised_color='red', thr=0.8, cmap='gray')
        post_str = str(key) + '_' + os.path.basename(im_fname) + '.npy'    
        cnm_As_fn = 'cnm_A_key' +  post_str
        np.save(os.path.join(analysis_dir,cnm_As_fn), cnm.estimates.A.todense())
        cnm_C_fn = 'cnm_C_key' +  post_str
        np.save(os.path.join(analysis_dir,cnm_C_fn), cnm.estimates.C)
        cnm_S_fn = 'cnm_S_key' +  post_str
        np.save(os.path.join(analysis_dir,cnm_S_fn), cnm.estimates.S)
        cnm.save(os.path.join(analysis_dir,'cnm_Obj_key' +  post_str + '.hdf5'))    

        # save data
        cn_filter_fn =  'cn_filter_key' + post_str 
        pnr_filter_fn = 'pnr_filter_key' + post_str
        np.save(os.path.join(analysis_dir,cn_filter_fn), cn_filter)
        np.save(os.path.join(analysis_dir,pnr_filter_fn), pnr)
        tff.imsave(os.path.join(analysis_dir,cn_filter_fn+'.tif'), cn_filter)
        tff.imsave(os.path.join(analysis_dir,pnr_filter_fn+'.tif'), pnr)

        toc = time.clock()
        print('Time taken:', toc-tic)


        #clean up
        min_SNR = 3            # adaptive way to set threshold on the transient size
        r_values_min = 0.85    # threshold on space consistency (if you lower more components
        #                        will be accepted, potentially with worst quality)
        cnm.params.set('quality', {'min_SNR': min_SNR,
                                   'rval_thr': r_values_min,
                                   'use_cnn': False})

        cnm.estimates.evaluate_components(Y, cnm.params)

        print(' ***** ')
        print('Number of total components: ', len(cnm.estimates.C))
        print('Number of accepted components: ', len(cnm.estimates.idx_components))

        idx = cnm.estimates.idx_components

        cnm_idx_fn = 'cnm_idx_key' + post_str
        np.save(os.path.join(analysis_dir, cnm_idx_fn), idx)
    except:
        print('Error in', key)
        f.write('\nError in key ' + str(key))
        f.close()
        if 'dview' in locals():
            cm.stop_server(dview=dview)
        pass
